In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time 

In [2]:
def createDFfromTable(url):
    # Deshabilitar la ventana de Google Chrome de WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    
    # Instanciamos el objeto WebDriver y accedemos a la url
    # Con ChromeDriverManager() instalamos la ultima versión compatible del webdriver de Chrome a través del repositorio oficial
    chrome_driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)    
    chrome_driver.get(url)
    
    # Contamos la cantidad de elementos dentro del "span" de la paginación de la tabla
    span_navigator_table = chrome_driver.find_element_by_class_name('ui-paginator-pages')
    indexs_navigation = span_navigator_table.find_elements_by_tag_name('span')
    indexs_navigation = [i + 1 for i in range(1, len(indexs_navigation))]

    # Si existe mas de un elemento (es decir, mas de una página)
    if len(indexs_navigation) > 1:
        
        # Creamos el DF inicial con el contenido de la tabla
        table_remuneraciones = chrome_driver.find_element_by_class_name('ui-datatable-tablewrapper').get_attribute('innerHTML')
        df_remuneraciones = pd.read_html(table_remuneraciones)[0]

        # Por cada elemento dentro del "span" 
        for index in indexs_navigation:
            
            # Obtenemos dicho elemento y simulamos un click, luego esperamos 2 segundos para que se refresque la tabla
            xpath = '//*[text()="' + str(index) + '"]'
            button = chrome_driver.find_element_by_xpath(xpath)
            button.click()
            time.sleep(2)
            
            # Creamos un DF temporal con el contenido de la nueva tabla generada y la mezclamos al primer DF (con el metodo .append())
            table_remuneraciones_temp = chrome_driver.find_element_by_class_name('ui-datatable-tablewrapper').get_attribute('innerHTML')
            df_remuneraciones_temp = pd.read_html(table_remuneraciones_temp)[0]
            df_remuneraciones = df_remuneraciones.append(df_remuneraciones_temp)

    #Si la tabla solo tiene una página
    else:    
        # Creamos un DF con el contenido de la tabla
        table_remuneraciones = chrome_driver.find_element_by_class_name('ui-datatable-tablewrapper').get_attribute('innerHTML')
        df_remuneraciones = pd.read_html(table_remuneraciones)[0]
    
    return df_remuneraciones

In [3]:
%matplotlib inline
df_remuneraciones = createDFfromTable('https://www.portaltransparencia.cl/PortalPdT/pdtta/-/ta/MU078/PR/PPLANT/50850666')
df_remuneraciones

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\Nico .SNOW\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache


,Año,Mes,Estamento,Nombre completo,Cargo o función,Grado EUS / Cargo con jornada,Calificación profesional o formación,Región,Asignaciones especiales,Unidad monetaria,Remuneración bruta mensualizada,Remuneración líquida mensualizada,Montos y horas extraordinarias diurnas,Montos y horas extraordinarias nocturnas,Montos y horas extraordinarias festivas,Fecha de inicio dd/mm/aa,Fecha de término dd/mm/aa,Observaciones,Declaración de intereses y patrimonio (publicación por buena práctica),Viáticos
0,2021,Junio,Auxiliar,"Arancibia Gonzalez, Victor Manuel",AUXILIAR - DAF,18,Enseñanza Media Completa,Región del Maule,(54)(13)(12),Pesos,553.718,213.183,"$28.216 : 14,00hrs","$31.441 : 13,00hrs",No tiene,01/02/2020,Indefinido,Sin observaciones,No publica,No informa
1,2020,Junio,Auxiliar,"Abarca Ugalde, Iván Alberto",GUARDIA - OFICINA SEGURIDAD INTERNA,13,Enseñanza Media Completa,Región del Maule,(41)(54)(13)(12),Pesos,1.046.309,748.299,"$3.396 : 1,00hrs","$162.994 : 40,00hrs",No tiene,23/01/2012,Indefinido,Sin observaciones,No publica,No informa
2,2020,Junio,Técnico,"Abrigo Robledo, Mary Carmen",REMUNERACIONES - DIRECCION GESTION DE PERSONAS,13,Secretaria Gerencial con mención Informatica,Región del Maule,(41),Pesos,881.967,367.452,No tiene,No tiene,No tiene,01/04/2000,Indefinido,Sin observaciones,No publica,No informa
3,2020,Junio,Profesional,"Acevedo Escobar, Mauricio Andres",PREVENCIONISTA DE RIESGOS,10,Ingeniero en Prevención de Riesgos y Medio Amb...,Región del Maule,(41)(30)(12),Pesos,1.540.466,1.248.433,"$45.806 : 8,00hrs",No tiene,No tiene,01/01/2017,Indefinido,Sin observaciones,No publica,No informa
4,2020,Junio,Técnico,"Agüero Ferreira, Cesar Alejandro","TECNICO - DIRECCION DE ASEO Y ORNATO, TRANSPOR...",14,Tecnico Nivel Medio en Produtos de la Madera,Región del Maule,(01),Pesos,676.285,474.039,No tiene,No tiene,No tiene,01/02/2020,Indefinido,Sin observaciones,No publica,No informa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2020,Junio,Técnico,"Villarroel Moraga, María Fabiola",SECRETARIA - DIRECCIÓN ADMINISTRACIÓN Y FINANZAS,11,Secretaria Ejecutiva mencion Computacion - Tec...,Región del Maule,(41),Pesos,1.087.140,367.825,No tiene,No tiene,No tiene,01/09/1995,Indefinido,Sin observaciones,No publica,No informa
66,2020,Junio,Auxiliar,"Villarroel Vergara, Cristian Marcelo",RADIO OPERADOR - COMITÉ COMUNAL DE EMERGENCIA,14,Enseñanza Media Completa,Región del Maule,(54)(13)(33),Pesos,1.040.237,864.509,No tiene,"$281.185 : 80,00hrs",No tiene,01/01/2005,Indefinido,Sin observaciones,No publica,No informa
67,2020,Junio,Técnico,"Villegas Escudero, Natalia Arlette",ATENCION DE PARVULO - SALA CUNA JARDIN PERSONITA,15,Teccnico - Atencion de Parvulo,Región del Maule,(54)(41),Pesos,643.257,532.263,No tiene,No tiene,No tiene,01/02/2020,Indefinido,Sin observaciones,No publica,No informa
68,2020,Junio,Jefatura,"Yañez Pol, Oscar Humberto",JEFE - DEPARTAMENTO ORGANIZACIONES COMUNITARIAS,8,Técnico Administración en Comercio Exterior,Región del Maule,(01),Pesos,1.808.928,1.617.372,No tiene,No tiene,No tiene,18/04/1988,Indefinido,Sin observaciones,No publica,No informa


In [4]:
# Se puede exportar directamente un Pandas DF a .csv con el metodo .to_csv()
# aunque se puede hacer de inmediato, es muy recomendable procesar el DF para remover acentos, eliminar caracteres no deseados
# y en general para estandarizar la información antes de insertar en la DB o guardar el .CSV
df_remuneraciones.to_csv('tabla_de_prueba.csv')